In [2]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
# import cv2
import numpy as np 
import pandas as pd
import re
import urllib
import pickle
import query
import re
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from fuzzywuzzy import fuzz

%load_ext autoreload
%autoreload 2
%aimport query

In [191]:
games_dict = open_pickle('giant_bomb_updated_game_info_new.pkl')
df = pd.DataFrame(games_dict).T.reset_index()

In [194]:
df

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra
0,Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,NaN
1,TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,NaN
2,Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,NaN
3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,"Action,Platformer",None,Knack,NaN
4,Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision,"Role-Playing,First-Person Shooter",Sci-Fi,Destiny,"Activision,Sony Interactive Entertainment,Bungie"
...,...,...,...,...,...,...,...,...,...
2007,Resident Evil: Revelations (console version),Resident Evil: Revelations,"January 26, 2012",Capcom,Capcom,"Shooter,Action-Adventure",Horror,Resident Evil,NaN
2008,Dead Nation (PS Vita),Dead Nation,"November 30, 2010",Housemarque,Sony Interactive Entertainment America,"Shooter,Dual-Joystick Shooter",Horror,None,"Sony Interactive Entertainment America,Sony In..."
2009,Disgaea 1 Complete,Disgaea: Hour of Darkness,"January 30, 2003",Nippon Ichi Software Inc.,Nippon Ichi Software Inc.,"Strategy,Role-Playing",Fantasy,Disgaea,"Nippon Ichi Software Inc.,Koei,Atlus U.S.A., Inc."
2010,Code Name: S.T.E.A.M.,None,"March 13, 2015",Nintendo SPD Group No.2,Nintendo,"Action,Strategy",Alternate Historical,None,NaN


In [132]:
games_dict_2 = open_pickle('giant_bomb_updated_game_info_new.pkl')
df_2 = pd.DataFrame(games_dict_2).T.reset_index()

In [133]:
df_2.iloc[1050:]

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra


In [129]:
titles = pd.read_sql(db_table_name,con=db_connection)['index'].to_list()

In [131]:
len(titles)

1050

In [180]:
db_connection = query.get_db_connection('../../../.supreme_db_creds.pkl',
                                            'Sandbox_Elena_Morais',
                                            dbhost='centaur')
db_table_name = 'Giantbomb_vgt_title_publisher_updates_new'


df.to_sql(db_table_name,con=db_connection,schema='Sandbox_Elena_Morais',if_exists='append')

In [75]:
# dict_new_2 = open_pickle('giant_bomb_updated_game_info_new.pkl')
# df_2 = pd.DataFrame(dict_new_2).T.reset_index()

# def is_li(val):
#     if type(val) == type([]):
#         return 1
#     else:
#         return 0
    
# df_2['dev_is_li'] = df_2.Developer.apply(lambda val: is_li(val))
# df_2['pub_is_li'] = df_2.Publisher.apply(lambda val: is_li(val))
# df_2['gen_is_li'] = df_2.Genre.apply(lambda val: is_li(val))
# df_2['theme_is_li'] = df_2.Theme.apply(lambda val: is_li(val))
# df_2['fran_is_li'] = df_2.Franchise.apply(lambda val: is_li(val))
# df_2['pub_e_is_li'] = df_2.Publisher_extra.apply(lambda val: is_li(val))

In [83]:
pd.DataFrame(dict_new).T.reset_index()

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra
0,Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,NaN
1,TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,NaN
2,Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,NaN
3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,"Action,Platformer",None,Knack,NaN
4,Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision,"Role-Playing,First-Person Shooter",Sci-Fi,Destiny,"Activision,Sony Interactive Entertainment,Bungie"
...,...,...,...,...,...,...,...,...,...
945,The Sims 4: Get to Work,The Sims 4,"September 2, 2014",Maxis Software,Electronic Arts,Simulation,Dating,The Sims,NaN
946,Little Friends: Dogs & Cats,Little Friends: Dogs & Cats,"May 27, 2019","Imagineer Co., Ltd.","Imagineer Co., Ltd.",Simulation,None,None,"Imagineer Co., Ltd.,Arc System Works Co., Ltd...."
947,Norn9: Var Commons,Norn9: Var Commons,"May 30, 2013",Otomate,Idea Factory,Adventure,Fantasy,None,"Idea Factory,Aksys Games"
948,LEGO The Lord of the Rings,LEGO The Lord of the Rings,"October 30, 2012",TT Games,WB Games,Action-Adventure,Fantasy,LEGO,NaN


In [84]:
# dict_new = open_pickle('giant_bomb_updated_game_info_new.pkl')
save_pickle('giant_bomb_updated_game_info_new.pkl',dict_new)

'giant_bomb_updated_game_info_new.pkl is saved'

In [65]:
df = pd.DataFrame(dict_new).T

In [66]:
df

,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra
Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,NaN
TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,NaN
Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,NaN
Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,"Action,Platformer",None,Knack,NaN
Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision,"Role-Playing,First-Person Shooter",Sci-Fi,Destiny,"Activision,Sony Interactive Entertainment,Bungie"
...,...,...,...,...,...,...,...,...
The Sims 4: Get to Work,The Sims 4,"September 2, 2014",Maxis Software,Electronic Arts,Simulation,Dating,The Sims,NaN
Little Friends: Dogs & Cats,Little Friends: Dogs & Cats,"May 27, 2019","Imagineer Co., Ltd.","Imagineer Co., Ltd.",Simulation,None,None,"Imagineer Co., Ltd.,Arc System Works Co., Ltd...."
Norn9: Var Commons,Norn9: Var Commons,"May 30, 2013",Otomate,Idea Factory,Adventure,Fantasy,None,"Idea Factory,Aksys Games"
LEGO The Lord of the Rings,LEGO The Lord of the Rings,"October 30, 2012",TT Games,WB Games,Action-Adventure,Fantasy,LEGO,NaN


In [114]:
df.reset_index(inplace=True)

In [115]:
df

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra
0,Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,NaN
1,TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,NaN
2,Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,NaN
3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,"Action,Platformer",None,Knack,NaN
4,Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision,"Role-Playing,First-Person Shooter",Sci-Fi,Destiny,"Activision,Sony Interactive Entertainment,Bungie"
...,...,...,...,...,...,...,...,...,...
945,The Sims 4: Get to Work,The Sims 4,"September 2, 2014",Maxis Software,Electronic Arts,Simulation,Dating,The Sims,NaN
946,Little Friends: Dogs & Cats,Little Friends: Dogs & Cats,"May 27, 2019","Imagineer Co., Ltd.","Imagineer Co., Ltd.",Simulation,None,None,"Imagineer Co., Ltd.,Arc System Works Co., Ltd...."
947,Norn9: Var Commons,Norn9: Var Commons,"May 30, 2013",Otomate,Idea Factory,Adventure,Fantasy,None,"Idea Factory,Aksys Games"
948,LEGO The Lord of the Rings,LEGO The Lord of the Rings,"October 30, 2012",TT Games,WB Games,Action-Adventure,Fantasy,LEGO,NaN


In [116]:
df[df['index'] == 'Splatoon 2']

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra


In [85]:
df_new = df.iloc[500:]

In [88]:
df_new.iloc[450:]

,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra


In [87]:
db_connection = query.get_db_connection('../../../.supreme_db_creds.pkl',
                                            'Sandbox_Elena_Morais',
                                            dbhost='centaur')
db_table_name = 'Giantbomb_vgt_title_publisher_updates_new'


df_new.to_sql(db_table_name,con=db_connection,schema='Sandbox_Elena_Morais',if_exists='append')

In [64]:
for i in dict_new.keys():
    if 'Publisher_extra' in dict_new[i].keys():
        if type(dict_new[i]['Publisher_extra']) == type([]):
            dict_new[i]['Publisher_extra'] = ','.join(dict_new[i]['Publisher_extra'])
        else:
            pass

In [3]:
import random

def get_data_from_page(rows,games_dict):
    game_d = {}
    for row in rows:
        for lab in ['Name','First release date','Platfrom',
                    'Developer','Publisher','Genre','Theme',
                    'Franchise','Aliasses']:
            if lab in row.text:
                if lab == 'First release date':
                    game_d[lab] = row.find('td').find('span').text
                elif lab == 'Genre' or lab == 'Platfrom':
                    if len(row.find('td').findAll('a')) > 1:
                        game_d[lab] = ','.join([i.text for i in row.find('td').findAll('a')])
                    elif row.find('td').find('a') == None:
                        game_d[lab] = None
                    else:
                        game_d[lab] = row.find('td').find('a').text    
                elif lab == 'Publisher':
                    if len(row.find('td').findAll('a')) > 1:
                        game_d['Publisher_extra'] = [i.text for i in row.find('td').findAll('a')]
                        game_d[lab] = row.find('td').find('a').text
                    elif row.find('td').find('a') == None:
                        game_d[lab] = None
                    else:
                        game_d[lab] = row.find('td').find('a').text
                else:
                    try:
                        game_d[lab] = row.find('td').find('a').text
                    except:
                        game_d[lab] = None
    return game_d

def get_game_details(title,driver,games_dict):
    t = title.split('(')[0]
    time.sleep(random.randint(0,2))
#     driver.find_element_by_partial_link_text(t).click()
    search_results = driver.find_elements_by_class_name('media')
    search_results[0].click()
    time.sleep(random.randint(0,2))
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    rows = soup.findAll('tr')
    games_dict[title] = get_data_from_page(rows,games_dict)
    return 'done'

def get_game_details_2(title,driver,games_dict):
    time.sleep(random.randint(0,2))
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    rows = soup.findAll('tr')
    games_dict[title] = get_data_from_page(rows,games_dict)
    return 'done'

In [508]:
DB_CONNECTION = query.get_db_connection('../../../.supreme_db_creds.pkl','Sandbox_Elena_Morais',dbhost='centaur')

In [7]:
all_tables = {}

for region in ['USA','EU']:
    for yr in range(2013,2021):
        title = f'{region}_VGT_{yr}_Table_update'
        all_tables[title] = query.return_table(DB_CONNECTION,'Sandbox_Elena_Morais',title)

In [11]:
tabs = []
for tab in all_tables.keys():
    tabs.append(all_tables[tab][['title','publisher','franchise_update']])

In [12]:
full_table = pd.concat(tabs)

In [18]:
titles = list(set(full_table.title.to_list()))

In [31]:
def save_pickle(file_name,obj):
    with open(file_name, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return file_name + ' is saved'

def open_pickle(file_name):
    with open(file_name, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

In [140]:
save_pickle('set_of_titles.pkl',titles)

'set_of_titles.pkl is saved'

In [50]:
titles = open_pickle('set_of_titles.pkl')

In [5]:
db_connection = query.get_db_connection('../../../.supreme_db_creds.pkl',
                                        'Sandbox_Elena_Morais',
                                        dbhost='centaur')

db_table_name = 'Giantbomb_vgt_title_publisher_updates_new'

titles = open_pickle('set_of_titles.pkl')

games_dict = {}

driver = webdriver.Chrome(executable_path='/Users/elena.morais/Documents/chromedriver_2') 
driver.get('https://www.giantbomb.com')
print('stopped after getting to page')

time.sleep(15)
print('stopped after getting to sleep')

c = 0
lower = 0

stopped after getting to page
stopped after getting to sleep


In [6]:
for title in titles:
    print(title)

    time.sleep(random.randint(2,5))
    driver.find_elements_by_xpath('//div[@class="js-masthead-toggle masthead-toggle masthead-toggle-search"]')[1].click()
    time.sleep(random.randint(0,2))
    driver.find_elements_by_xpath("//option[@value='game']")[0].click()
    driver.find_element_by_xpath('//input[@class="js-site-search-query"]').send_keys(title)
    time.sleep(random.randint(0,2))
#     driver.find_element_by_xpath('//select[@class="dropdown--selector"]').click()
#     driver.find_element_by_xpath('//option[@value="game"]').click()
    time.sleep(random.randint(0,2))
    driver.find_element_by_xpath('//input[@class="js-site-search-query"]').send_keys(u'\ue007')
#     river-search__select
    get_game_details(title,driver,games_dict)
    c += 1 

    if c % 500 == 0:
        save_df = pd.DataFrame(games_dict).iloc[lower:c,:]
        lower = c
        save_df.to_sql(db_table_name,con=db_connection,schema='Sandbox_Elena_Morais',if_exists='append')

    else:
        pass

save_df = pd.DataFrame(games_dict).iloc[lower:,:]
save_df.to_sql(db_table_name,con=db_connection,schema='Sandbox_Elena_Morais',if_exists='append')

Dragon Quest XI S: Echoes of an Elusive Age - D


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=87.0.4280.141)


In [200]:
driver = webdriver.Chrome(executable_path='/Users/elena.morais/Documents/chromedriver_2') 
driver.get('https://www.giantbomb.com')

time.sleep(30)

In [262]:
c = 0 
for title in titles[1978:]:
    print(title)

    time.sleep(random.randint(2,5))
    driver.find_elements_by_xpath('//div[@class="js-masthead-toggle masthead-toggle masthead-toggle-search"]')[1].click()
    time.sleep(random.randint(0,2))
    driver.find_elements_by_xpath("//option[@value='game']")[0].click()
    driver.find_element_by_xpath('//input[@class="js-site-search-query"]').send_keys(title)
    time.sleep(random.randint(0,2))
#     driver.find_element_by_xpath('//select[@class="dropdown--selector"]').click()
#     driver.find_element_by_xpath('//option[@value="game"]').click()
    time.sleep(random.randint(0,2))
    driver.find_element_by_xpath('//input[@class="js-site-search-query"]').send_keys(u'\ue007')
#     river-search__select
    get_game_details(title,driver,games_dict)
    
    if c%20 == 0:
        save_pickle('giant_bomb_updated_game_info.pkl',games_dict)
        

save_pickle('giant_bomb_updated_game_info.pkl',games_dict)
        

Pokemon Sword: Expansion Pass (The Isle of Armor / The Crown Tundra)
Don Bradman Cricket 14
Hyperdimension Neptunia: Re;Birth3 - V Generati
Danganronpa Another Episode: Ultra Despair Girl
Adventure Time: The Secret of the Nameless Kingdom
Elite: Dangerous
Shin Megami Tensei: Strange Journey Redux
FIFA Soccer 13
Ni no Kuni: Wrath of the White Witch for Nintendo Switch
Payday 3
Marvel's Avengers
Soul Sacrifice
Steins;Gate 0
Chaos;Child
Nike+ Kinect Training
F1 2020
Adventure Time: Pirates of the Enchiridion
Penny Punching Princess
Astroneer
Crusader Kings III
PixelJunk Monsters 2
Knowledge is Power
Rise of the Guardians: The Video Game
Port Royale 4
Bladestorm: Nightmare
BlazBlue: Chrono Phantasma Extend
The Trash Pack
The Legend of Zelda: Tri Force Heroes
Buildings Have Feelings Too
Puzzles & Dragons Z + Puzzles & Dragons Super Mario Bros. Edition
Disney Magical World 2
Batman: A Telltale Games Series
NHL 17
Nights of Azure 2: Bride of the New Moon
Dr. Kawashima's Devilish Brain Trainin

In [267]:
df = pd.DataFrame.from_dict(games_dict, orient='index')

In [276]:
df = df.reset_index().rename(columns={'index':'vgt_name'})

In [270]:
df.Publisher.isnull().sum()

31

In [355]:
def get_fuzzy_match(row):
    name = row['vgt_name']
    name1 = row['Name']
    return fuzz.token_set_ratio(name, name1)

def get_fuzzy_match_2(row):
    name = row['clean_title']
    name1 = row['Name']
    return fuzz.token_set_ratio(name, name1)

In [273]:
df.index

Index(['Dragon Quest XI S: Echoes of an Elusive Age - D', 'TANK! TANK! TANK!',
       'Road Rage', 'Knack 2', 'Destiny: The Taken King', 'The Banner Saga 2',
       'The Witch and the Hundred Knight', 'Code of Princess',
       'Hatsune Miku: Project Mirai DX', 'Code: Realize - Guardian of Rebirth',
       ...
       'Harvest Moon: A New Beginning', 'Grounded', 'Pro Cycling Manager 2015',
       'Atelier Ryza: Ever Darkness & the Secret Hideout', 'Starfield',
       'Dark Rose Valkyrie', 'Dead Space 3', 'Olympic Games Tokyo 2020',
       'Windbound', 'Titanfall 2'],
      dtype='object', length=2030)

In [280]:
df['fuzzy_ratio'] = df.apply(get_fuzzy_match, axis=1)

In [311]:
# df.sort_values("fuzzy_ratio", ascending = True)
df[df['fuzzy_ratio'] < 90].iloc[55:,:]

### hard code updates:

'Disgaea 5 Complete':

Happy Birthdays : https://www.nintendo.com/games/detail/happy-birthdays-switch/
Inazuma Eleven 3: Team Ogre Attacks: https://www.metacritic.com/game/3ds/inazuma-eleven-3-team-ogre-attacks!/details
'Sherlock Holmes and the Mystery of the Frozen C''Sherlock Holmes and the Mystery of the Frozen City':https://www.metacritic.com/game/3ds/sherlock-holmes-and-the-mystery-of-the-frozen-city
Imagine: Fashion Life: https://www.nintendo.com/games/detail/imagine-fashion-life-3ds/
The Elder Scrolls Online: Murkmire: https://elderscrolls.fandom.com/wiki/The_Elder_Scrolls_Online:_Murkmire,
The Elder Scrolls Online: Summerset: https://elderscrolls.fandom.com/wiki/The_Elder_Scrolls_Online:_Summerset
The Elder Scrolls Online: Greymoor: https://store.steampowered.com/app/1205430/The_Elder_Scrolls_Online__Greymoor/
Resident Evil: Resistance:
Inazuma Eleven 3: Bomb Blast: https://www.metacritic.com/game/3ds/inazuma-eleven-3-bomb-blast
Inazuma Eleven 3: Lightning Bolt: 
New Style Boutique: https://www.giantbomb.com/style-savvy-trendsetters/3030-39284/
New Style Boutique 3: Styling Star: https://www.giantbomb.com/style-savvy-styling-star/3030-61367/
Attack on Titan: Wings of Freedom: https://www.giantbomb.com/attack-on-titan/3030-50516/
Jewel Quest: The Sapphire Dragon: https://nintendo.fandom.com/wiki/Jewel_Quest:_The_Sapphire_Dragon
Hitman: Definitive Edition: https://www.metacritic.com/game/playstation-4/hitman-definitive-edition/details
Mechs vs. Minions: https://leagueoflegends.fandom.com/wiki/Mechs_vs._Minions
European Ship Simulator: https://store.steampowered.com/app/299250/European_Ship_Simulator/
Puzzler: Brain Games: https://www.nintendolife.com/reviews/3ds/puzzler_brain_games
Payday 3: https://www.ign.com/games/payday-3












Dr. Kawashima's Devilish Brain Training: Can You Stay Focused: https://www.nintendo.com.au/catalogue/dr-kawashimas-devilish-brain-training-can-you-stay-focused


In [152]:
hand_done_titles = {'Happy Birthdays':{'Name':'Happy Birthdays','First release date':'Jun 05, 2018',
                    'Platfrom':'Switch','Developer':['Arc System Works', 'TOYBOX Inc.'],
                    'Publisher':'NIS America Inc.','Publisher_extra':None,'Genre':'Simulation','Theme':None,
                    'Franchise':None,'Aliasses':None},
'Inazuma Eleven 3: Team Ogre Attacks':{'Name':'Inazuma Eleven 3: Team Ogre Attacks!',
                                       'First release date':'Feb 14, 2014',
                                        'Platfrom':'3DS,DS','Developer':'Level 5',
                                        'Publisher':'Level 5','Publisher_extra':None,'Genre':['Sports', 'Traditional', 
                                                                        'Soccer', 'Arcade'],
                                       'Theme':None,'Franchise':'Inazuma Eleven 3','Aliasses':None},
'Sherlock Holmes and the Mystery of the Frozen C':{'Name':'Sherlock Holmes and the Mystery of the Frozen City',
                                                   'First release date':'Q2 2012','Platfrom':'3DS',
                                                   'Developer':'Frogwares','Publisher':'Focus Home Interactive',
                                                   'Publisher_extra':None,'Genre':['Action Adventure', 'Fantasy'],
                                                   'Theme':None,
                                                   'Franchise':'Sherlock Holmes','Aliasses':None},
 'Sherlock Holmes and the Mystery of the Frozen City':{'Name':'Sherlock Holmes and the Mystery of the Frozen City',
                                                       'First release date':'Q2 2012','Platfrom':'3DS',
                                                       'Developer':'Frogwares','Publisher':'Focus Home Interactive',
                                                       'Publisher_extra':None,
                                                       'Genre':['Action Adventure', 'Fantasy'],'Theme':None,
                                                       'Franchise':'Sherlock Holmes','Aliasses':None},
 'Imagine: Fashion Life':{'Name':'Imagine Fashion Life','First release date':'May 8, 2012',
                          'Platfrom':['Nintendo DS', 'Nintendo 3DS', 'Wii', 'Microsoft Windows'],
                          'Developer':None,'Publisher':'Ubisoft','Publisher_extra':None,'Genre':'Simulation','Theme':'Fsshion',
                          'Franchise':'Imagine','Aliasses':None},
  'The Elder Scrolls Online: Greymoor':{'Name':'The Elder Scrolls Online - Greymoor',
                                        'First release date':'May 26, 2020',
                                        'Platfrom':['PC','PlayStation 4','Xbox One'],'Developer':'Zenimax Online Studios',
                                        'Publisher':'Bethesda Softworks','Publisher_extra':None,
                                        'Genre':['Action','RPG'],'Theme':None,
                                        'Franchise':'The Elder Scrolls Online','Aliasses':None},
  'The Elder Scrolls Online: Murkmire':{'Name':'','First release date':'October 22, 2018',
                    'Platfrom':['PC','PlayStation 4','Xbox One'],'Developer':'Zenimax Online Studios',
                    'Publisher':'Bethesda Softworks','Publisher_extra':None,'Genre':['Action','RPG'],'Theme':None,
                    'Franchise':'The Elder Scrolls Online','Aliasses':None},
  'The Elder Scrolls Online: Summerset':{'Name':'','First release date':'April 16th, 2018',
                    'Platfrom':['PC','PlayStation 4','Xbox One'],'Developer':'Zenimax Online Studios',
                    'Publisher':'Bethesda Softworks','Publisher_extra':None,'Genre':['Action','RPG'],'Theme':None,
                    'Franchise':'The Elder Scrolls Online','Aliasses':None},
  'Resident Evil: Resistance':{'Name':'Resident Evil Resistance','First release date':None,
                               'Platfrom':'PC','Developer':['CAPCOM Co., Ltd.', 'NeoBards Entertainment Ltd.'],
                               'Publisher':'CAPCOM Co., Ltd.','Publisher_extra':None,'Genre':'Action','Theme':'Zombies',
                               'Franchise':'Resident Evil','Aliasses':None},
  'Inazuma Eleven 3: Bomb Blast':{'Name':'Inazuma Eleven 3: Bomb Blast','First release date':'Sep 27, 2013',
                               'Platfrom':'3DS,DS','Developer':'Level 5',
                               'Publisher':'Level 5','Publisher_extra':None,'Genre':'Sports, RPG, Soccer, Arcade','Theme':None,
                               'Franchise':'Inazuma Eleven 3','Aliasses':None},
 'Inazuma Eleven 3: Lightning Bolt':{'Name':'Inazuma Eleven 3: Lightning Bolt','First release date':'Jul 01, 2010',
                               'Platfrom':'3DS,DS','Developer':'Level 5',
                               'Publisher':'Level 5','Publisher_extra':None,'Genre':'Sports, RPG, Soccer, Arcade','Theme':None,
                               'Franchise':'Inazuma Eleven 3','Aliasses':None},
 'New Style Boutique':{'Name':'Style Savvy: Trendsetters','First release date':'September 27, 2012',
                       'Platfrom':'3DS','Developer':'syn Sophia,Nintendo SPD',
                       'Publisher':'Nintendo','Publisher_extra':None,'Genre':'Simulation',
                       'Theme':'fashion,Management','Franchise':'New Style Boutique',
                       'Aliasses':'Wagamama Fashion: Girls Mode – Greed Declaration,Style Savvy 3DS,Nintendo presents: New Style Boutique,Girls Style: Yokubari Sengen!'},
 'New Style Boutique 3: Styling Star':{'Name':'Style Savvy: Styling Star',
                                       'First release date':'November 2, 2017',
                                       'Platfrom':'3DS','Developer':'syn Sophia',
                                       'Publisher':'Nintendo','Publisher_extra':None,'Genre':'Simulation','Theme':'fashion,Management',
                                       'Franchise':'New Style Boutique',
                                       'Aliasses':'Girls Mode 4 Star ☆ Stylist,Girls Mode 4 スター☆スタイリスト,Style Savvy 4,Nintendo presents: New Style Boutique 3 - Styling Star'},
 'Attack on Titan: Wings of Freedom':{'Name':'Attack on Titan',
                                      'First release date':'February 18, 2016',
                               'Platfrom':'PlayStation 3,PlayStation Network (PS3),PC,PlayStation Vita,PlayStation Network (Vita),Xbox One,PlayStation 4',
                                      'Developer':'Omega Force',
                               'Publisher':'KOEI TECMO GAMES CO., LTD.','Publisher_extra':None,'Genre':'Action','Theme':'Anime,Horror',
                               'Franchise':'Attack on Titan','Aliasses':'A.O.T. Wings of Freedom'},
  'Jewel Quest: The Sapphire Dragon':{'Name':'Jewel Quest: The Sapphire Dragon','First release date':'10/20/2011',
                               'Platfrom':'PC','Developer':'iWin',
                               'Publisher':'iWin','Publisher_extra':None,'Genre':'puzzle','Theme':None,
                               'Franchise':'Jewel Quest','Aliasses':None},
  'Hitman: Definitive Edition':{'Name':'Hitman: Definitive Edition','First release date':'May 15, 2018',
                               'Platfrom':'Xbox One,PS4','Developer':'Io Interactive',
                               'Publisher':'Warner Bros. Interactive Entertainment','Publisher_extra':None,
                                'Genre':'Action Adventure,','Theme':'',
                               'Franchise':None,'Aliasses':None},
  'Mechs vs. Minions':{'Name':'Mechs vs. Minions','First release date':None,
                       'Platfrom':'','Developer':'Riot Games Inc.,Stone Librande',
                       'Publisher':'Riot Games Inc.','Publisher_extra':None,'Genre':'Tabletop game',
                       'Theme':None,'Franchise':'League of Legends','Aliasses':None},
  'European Ship Simulator':{'Name':'European Ship Simulator','First release date':'Nov 4, 2016',
                               'Platfrom':'PC','Developer':'Excalibur',
                               'Publisher':'Excalibur Publishing','Publisher_extra':None,'Genre':'Simulation','Theme':None,
                               'Franchise':None,'Aliasses':None},
  'Puzzler: Brain Games':{'Name':'Puzzler: Brain Games','First release date':'April 23, 2013',
                               'Platfrom':'3DS','Developer':'Maximum Games',
                               'Publisher':'Nintendo','Publisher_extra':None,'Genre':'Puzzle','Theme':None,
                               'Franchise':'Puzzler','Aliasses':None},
  'Payday 3':{'Name':'Payday 3','First release date':None,
                               'Platfrom':None,'Developer':'Starbreeze AB',
                               'Publisher':None,'Publisher_extra':None,'Genre':'First-person shooter,Tactical shooter,Stealth','Theme':None,
                               'Franchise':'Payday','Aliasses':None},
  "Dr. Kawashima's Devilish Brain Training: Can Yo":{'Name':"Dr. Kawashima's Devilish Brain Training: Can You Stay Focused",
                                                     'First release date':None,
                                                     'Platfrom':'DS',
                                                     'Developer':'Namco Bandai',
                                                     'Publisher':'Nintendo',
                                                     'Publisher_extra':None,
                                                     'Genre':'Puzzle',
                                                     'Theme':None,
                                                     'Franchise':'Brain Training',
                                                     'Aliasses':None}
}

In [192]:

games_dict.update(hand_done_titles)

hand_done = pd.DataFrame(games_dict).T.reset_index()

In [201]:
d.columns

Index(['index', 'Name', 'First release date', 'Developer', 'Publisher',
       'Genre', 'Theme', 'Franchise', 'Publisher_extra', 'Platfrom',
       'Aliasses'],
      dtype='object')

In [202]:
d = hand_done.iloc[2012:]

def is_li(val):
    if type(val) == type([]):
        return ','.join(val)
    else:
        return val

d['index'] = d['index'].apply(lambda val: is_li(val))
d['Name'] = d['Name'].apply(lambda val: is_li(val))
d['Developer'] = d['Developer'].apply(lambda val: is_li(val))
d['Publisher'] = d['Publisher'].apply(lambda val: is_li(val))
d['Genre'] = d['Genre'].apply(lambda val: is_li(val))
d['Theme'] = d['Theme'].apply(lambda val: is_li(val))
d['Franchise'] = d['Franchise'].apply(lambda val: is_li(val))
d['Publisher_extra'] = d['Publisher_extra'].apply(lambda val: is_li(val))



# for col in d.columns:
    
d = d[['index', 'Name', 'First release date', 'Developer', 'Publisher',
       'Genre', 'Theme', 'Franchise', 'Publisher_extra']]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [204]:
def is_li_2(val):
    if type(val) == type([]):
        return 1
    else:
        return 0
    
d['dev_is_li'] = d.Developer.apply(lambda val: is_li_2(val))
d['pub_is_li'] = d.Publisher.apply(lambda val: is_li_2(val))
d['gen_is_li'] = d.Genre.apply(lambda val: is_li_2(val))
d['theme_is_li'] = d.Theme.apply(lambda val: is_li_2(val))
d['fran_is_li'] = d.Franchise.apply(lambda val: is_li_2(val))
d['pub_e_is_li'] = d.Publisher_extra.apply(lambda val: is_li_2(val))

In [208]:
d = d[['index', 'Name', 'First release date', 'Developer', 'Publisher',
       'Genre', 'Theme', 'Franchise', 'Publisher_extra']]

In [209]:
d

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra
2012,Happy Birthdays,Happy Birthdays,"Jun 05, 2018","Arc System Works,TOYBOX Inc.",NIS America Inc.,Simulation,None,None,None
2013,Inazuma Eleven 3: Team Ogre Attacks,Inazuma Eleven 3: Team Ogre Attacks!,"Feb 14, 2014",Level 5,Level 5,"Sports,Traditional,Soccer,Arcade",None,Inazuma Eleven 3,None
2014,Sherlock Holmes and the Mystery of the Frozen C,Sherlock Holmes and the Mystery of the Frozen ...,Q2 2012,Frogwares,Focus Home Interactive,"Action Adventure,Fantasy",None,Sherlock Holmes,None
2015,Sherlock Holmes and the Mystery of the Frozen ...,Sherlock Holmes and the Mystery of the Frozen ...,Q2 2012,Frogwares,Focus Home Interactive,"Action Adventure,Fantasy",None,Sherlock Holmes,None
2016,Imagine: Fashion Life,Imagine Fashion Life,"May 8, 2012",None,Ubisoft,Simulation,Fsshion,Imagine,None
2017,The Elder Scrolls Online: Greymoor,The Elder Scrolls Online - Greymoor,"May 26, 2020",Zenimax Online Studios,Bethesda Softworks,"Action,RPG",None,The Elder Scrolls Online,None
2018,The Elder Scrolls Online: Murkmire,,"October 22, 2018",Zenimax Online Studios,Bethesda Softworks,"Action,RPG",None,The Elder Scrolls Online,None
2019,The Elder Scrolls Online: Summerset,,"April 16th, 2018",Zenimax Online Studios,Bethesda Softworks,"Action,RPG",None,The Elder Scrolls Online,None
2020,Resident Evil: Resistance,Resident Evil Resistance,None,"CAPCOM Co., Ltd.,NeoBards Entertainment Ltd.","CAPCOM Co., Ltd.",Action,Zombies,Resident Evil,None
2021,Inazuma Eleven 3: Bomb Blast,Inazuma Eleven 3: Bomb Blast,"Sep 27, 2013",Level 5,Level 5,"Sports, RPG, Soccer, Arcade",None,Inazuma Eleven 3,None


In [210]:
# db_connection = query.get_db_connection('../../../.supreme_db_creds.pkl',
#                                             'Sandbox_Elena_Morais',
#                                             dbhost='centaur')
db_table_name = 'Giantbomb_vgt_title_publisher_updates_new'

d.to_sql(db_table_name,con=db_connection,schema='Sandbox_Elena_Morais',if_exists='append')

,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses
2012,Happy Birthdays,Happy Birthdays,"Jun 05, 2018","[Arc System Works, TOYBOX Inc.]",NIS America Inc.,Simulation,None,None,None,Switch,None
2013,Inazuma Eleven 3: Team Ogre Attacks,Inazuma Eleven 3: Team Ogre Attacks!,"Feb 14, 2014",Level 5,Level 5,"[Sports, Traditional, Soccer, Arcade]",None,Inazuma Eleven 3,None,"3DS,DS",None
2014,Sherlock Holmes and the Mystery of the Frozen C,Sherlock Holmes and the Mystery of the Frozen ...,Q2 2012,Frogwares,Focus Home Interactive,"[Action Adventure, Fantasy]",None,Sherlock Holmes,None,3DS,None
2015,Sherlock Holmes and the Mystery of the Frozen ...,Sherlock Holmes and the Mystery of the Frozen ...,Q2 2012,Frogwares,Focus Home Interactive,"[Action Adventure, Fantasy]",None,Sherlock Holmes,None,3DS,None
2016,Imagine: Fashion Life,Imagine Fashion Life,"May 8, 2012",None,Ubisoft,Simulation,Fsshion,Imagine,None,"[Nintendo DS, Nintendo 3DS, Wii, Microsoft Win...",None
2017,The Elder Scrolls Online: Greymoor,The Elder Scrolls Online - Greymoor,"May 26, 2020",Zenimax Online Studios,Bethesda Softworks,"[Action, RPG]",None,The Elder Scrolls Online,None,"[PC, PlayStation 4, Xbox One]",None
2018,The Elder Scrolls Online: Murkmire,,"October 22, 2018",Zenimax Online Studios,Bethesda Softworks,"[Action, RPG]",None,The Elder Scrolls Online,None,"[PC, PlayStation 4, Xbox One]",None
2019,The Elder Scrolls Online: Summerset,,"April 16th, 2018",Zenimax Online Studios,Bethesda Softworks,"[Action, RPG]",None,The Elder Scrolls Online,None,"[PC, PlayStation 4, Xbox One]",None
2020,Resident Evil: Resistance,Resident Evil Resistance,None,"[CAPCOM Co., Ltd., NeoBards Entertainment Ltd.]","CAPCOM Co., Ltd.",Action,Zombies,Resident Evil,None,PC,None
2021,Inazuma Eleven 3: Bomb Blast,Inazuma Eleven 3: Bomb Blast,"Sep 27, 2013",Level 5,Level 5,"Sports, RPG, Soccer, Arcade",None,Inazuma Eleven 3,None,"3DS,DS",None


In [189]:
hand_done = pd.DataFrame(hand_done_titles).T.reset_index()

In [190]:
hand_done

,index,Name,First release date,Platfrom,Developer,Publisher,Publisher_extra,Genre,Theme,Franchise,Aliasses
0,Happy Birthdays,Happy Birthdays,"Jun 05, 2018",Switch,"[Arc System Works, TOYBOX Inc.]",NIS America Inc.,None,Simulation,None,None,None
1,Inazuma Eleven 3: Team Ogre Attacks,Inazuma Eleven 3: Team Ogre Attacks!,"Feb 14, 2014","3DS,DS",Level 5,Level 5,None,"[Sports, Traditional, Soccer, Arcade]",None,Inazuma Eleven 3,None
2,Sherlock Holmes and the Mystery of the Frozen C,Sherlock Holmes and the Mystery of the Frozen ...,Q2 2012,3DS,Frogwares,Focus Home Interactive,None,"[Action Adventure, Fantasy]",None,Sherlock Holmes,None
3,Sherlock Holmes and the Mystery of the Frozen ...,Sherlock Holmes and the Mystery of the Frozen ...,Q2 2012,3DS,Frogwares,Focus Home Interactive,None,"[Action Adventure, Fantasy]",None,Sherlock Holmes,None
4,Imagine: Fashion Life,Imagine Fashion Life,"May 8, 2012","[Nintendo DS, Nintendo 3DS, Wii, Microsoft Win...",None,Ubisoft,None,Simulation,Fsshion,Imagine,None
...,...,...,...,...,...,...,...,...,...,...,...
2028,Resident Evil: Revelations (console version),Resident Evil: Revelations,"January 26, 2012",NaN,Capcom,Capcom,NaN,"Shooter,Action-Adventure",Horror,Resident Evil,NaN
2029,Dead Nation (PS Vita),Dead Nation,"November 30, 2010",NaN,Housemarque,Sony Interactive Entertainment America,"Sony Interactive Entertainment America,Sony In...","Shooter,Dual-Joystick Shooter",Horror,None,NaN
2030,Disgaea 1 Complete,Disgaea: Hour of Darkness,"January 30, 2003",NaN,Nippon Ichi Software Inc.,Nippon Ichi Software Inc.,"Nippon Ichi Software Inc.,Koei,Atlus U.S.A., Inc.","Strategy,Role-Playing",Fantasy,Disgaea,NaN
2031,Code Name: S.T.E.A.M.,None,"March 13, 2015",NaN,Nintendo SPD Group No.2,Nintendo,NaN,"Action,Strategy",Alternate Historical,None,NaN


In [174]:
hand_done.T.to_dict()

{'Happy Birthdays': {'Name': 'Happy Birthdays',
  'First release date': 'Jun 05, 2018',
  'Platfrom': 'Switch',
  'Developer': 'Arc System Works,TOYBOX Inc.',
  'Publisher': 'NIS America Inc.',
  'Publisher_extra': None,
  'Genre': 'Simulation',
  'Theme': None,
  'Franchise': None,
  'Aliasses': None},
 'Inazuma Eleven 3: Team Ogre Attacks': {'Name': 'Inazuma Eleven 3: Team Ogre Attacks!',
  'First release date': 'Feb 14, 2014',
  'Platfrom': '3DS,DS',
  'Developer': 'Level 5',
  'Publisher': 'Level 5',
  'Publisher_extra': None,
  'Genre': 'Sports,Traditional,Soccer,Arcade',
  'Theme': None,
  'Franchise': 'Inazuma Eleven 3',
  'Aliasses': None},
 'Sherlock Holmes and the Mystery of the Frozen C': {'Name': 'Sherlock Holmes and the Mystery of the Frozen City',
  'First release date': 'Q2 2012',
  'Platfrom': '3DS',
  'Developer': 'Frogwares',
  'Publisher': 'Focus Home Interactive',
  'Publisher_extra': None,
  'Genre': 'Action Adventure,Fantasy',
  'Theme': None,
  'Franchise': 'Sher

In [175]:
save_pickle('hand_done_titles.pkl',hand_done.T.to_dict())

'hand_done_titles.pkl is saved'

In [173]:
def is_li(val):
    if type(val) == type([]):
        return ','.join(val)
    else:
        return val
    
hand_done['Developer'] = hand_done.Developer.apply(lambda val: is_li(val))
hand_done['Publisher'] = hand_done.Publisher.apply(lambda val: is_li(val))
hand_done['Genre'] = hand_done.Genre.apply(lambda val: is_li(val))
hand_done['Theme'] = hand_done.Theme.apply(lambda val: is_li(val))
hand_done['Franchise'] = hand_done.Franchise.apply(lambda val: is_li(val))
hand_done['Publisher_extra'] = hand_done.Publisher_extra.apply(lambda val: is_li(val))

In [95]:
pages = {'Borderlands 2 (PS Vita)':'https://www.giantbomb.com/borderlands-2/3030-36055/',
 'Sonic Boom':'https://www.giantbomb.com/sonic-boom/3030-949/',
 'Pokemon X Version':'https://www.giantbomb.com/pokemon-xy/3030-41088/',
 "Madagascar 3: Europe's Most Wanted":'https://www.giantbomb.com/madagascar-3-the-video-game/3030-37449/',
 'Final Fantasy XV Windows Edition':'https://www.giantbomb.com/final-fantasy-xv/3030-21006/',
 'Dungeons & Dragons: Neverwinter':'https://www.giantbomb.com/neverwinter/3030-32417/',
 'Minecraft: Wii U Edition':'https://www.giantbomb.com/minecraft/3030-30475/',
 'DriveClub Bikes':'https://www.giantbomb.com/driveclub/3030-41693/',
 'Minecraft: PlayStation 3 Edition':'https://www.giantbomb.com/minecraft/3030-30475/',
 'Heavy Fire: The Chosen Few':'https://www.giantbomb.com/heavy-fire-afghanistan/3030-37010/',
 'Myst 3DS':'https://www.giantbomb.com/myst/3030-3970/',
 "Deadlight: Director's Cut":'https://www.giantbomb.com/deadlight/3030-37262/',
 'Sly Cooper Collection':"https://www.giantbomb.com/the-sly-collection/3030-31788/",
 'Minecraft: PS Vita Edition':"https://www.giantbomb.com/minecraft/3030-30475/",
 'Resident Evil: Revelations (console version)':"https://www.giantbomb.com/resident-evil-revelations/3030-31737/",
 'Dead Nation (PS Vita)':"https://www.giantbomb.com/dead-nation/3030-27866/",
 'Disgaea 1 Complete':"https://www.giantbomb.com/disgaea-hour-of-darkness/3030-18540/",
 'Code Name: S.T.E.A.M.':"https://www.giantbomb.com/code-name-s-t-e-a-m/3030-46597/",
 'Resident Evil Zero':"https://www.giantbomb.com/resident-evil-0/3030-10247/"}

In [17]:
ts = list(pages.keys())
t2 = list(hand_done_titles.keys())
ts.extend(t2)

In [18]:
ts

['Borderlands 2 (PS Vita)',
 'Sonic Boom',
 'Pokemon X Version',
 "Madagascar 3: Europe's Most Wanted",
 'Final Fantasy XV Windows Edition',
 'Dungeons & Dragons: Neverwinter',
 'Minecraft: Wii U Edition',
 'DriveClub Bikes',
 'Minecraft: PlayStation 3 Edition',
 'Heavy Fire: The Chosen Few',
 'Myst 3DS',
 "Deadlight: Director's Cut",
 'Sly Cooper Collection',
 'Minecraft: PS Vita Edition',
 'Resident Evil: Revelations (console version)',
 'Dead Nation (PS Vita)',
 'Disgaea 1 Complete',
 'Code Name: S.T.E.A.M.',
 'Resident Evil Zero',
 'Happy Birthdays',
 'Inazuma Eleven 3: Team Ogre Attacks',
 'Sherlock Holmes and the Mystery of the Frozen C',
 'Sherlock Holmes and the Mystery of the Frozen City',
 'Imagine: Fashion Life',
 'The Elder Scrolls Online: Greymoor',
 'The Elder Scrolls Online: Murkmire',
 'The Elder Scrolls Online: Summerset',
 'Resident Evil: Resistance',
 'Inazuma Eleven 3: Bomb Blast',
 'Inazuma Eleven 3: Lightning Bolt',
 'New Style Boutique',
 'New Style Boutique 3: S

In [310]:
driver = webdriver.Chrome(executable_path='/Users/elena.morais/Documents/chromedriver_2')

for title in pages.keys():
    print(title)
    driver.get(pages[title])

    time.sleep(random.randint(2,5))
    get_game_details_2(title,driver,games_dict)
    
save_pickle('giant_bomb_updated_game_info.pkl',games_dict)
    
# games_dict

Borderlands 2 (PS Vita)
Sonic Boom
Pokemon X Version
Madagascar 3: Europe's Most Wanted
Final Fantasy XV Windows Edition
Dungeons & Dragons: Neverwinter
Minecraft: Wii U Edition
DriveClub Bikes
Minecraft: PlayStation 3 Edition
Heavy Fire: The Chosen Few
Myst 3DS
Deadlight: Director's Cut
Sly Cooper Collection
Minecraft: PS Vita Edition
Resident Evil: Revelations (console version)
Dead Nation (PS Vita)
Disgaea 1 Complete
Code Name: S.T.E.A.M.
Resident Evil Zero


'giant_bomb_updated_game_info.pkl is saved'

In [319]:
games_dict.update(hand_done_titles)
df = pd.DataFrame.from_dict(games_dict, orient='index')
df = df.reset_index().rename(columns={'index':'vgt_name'})
df['fuzzy_ratio'] = df.apply(get_fuzzy_match, axis=1)

In [356]:
df['clean_title'] = df.vgt_name.apply(lambda title: title.split('(')[0])
df['fuzzy_ratio_2'] = df.apply(get_fuzzy_match_2, axis=1)

In [357]:
df[df.fuzzy_ratio_2 < 90]

,vgt_name,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses,fuzzy_ratio,clean_title,fuzzy_ratio_2
3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,[],None,Knack,NaN,NaN,NaN,83,Knack 2,83
62,Pokemon Moon,Pokémon Sun/Moon,"November 18, 2016","Game Freak, Inc.",Nintendo,Role-Playing,Fantasy,Pokémon,NaN,NaN,NaN,81,Pokemon Moon,81
81,"Pokemon: Let's Go, Pikachu!","Pokémon: Let's Go, Pikachu!/Eevee!","November 16, 2018","Game Freak, Inc.",The Pokémon Company,Role-Playing,Fantasy,Pokémon,NaN,NaN,NaN,87,"Pokemon: Let's Go, Pikachu!",87
136,Outlast II,Outlast 2,"April 25, 2017",Red Barrels,Red Barrels,Adventure,Horror,Outlast,NaN,NaN,NaN,88,Outlast II,88
145,Kirby and the Rainbow Paintbrush,Kirby and the Rainbow Curse,"January 22, 2015","HAL Laboratory, Inc.",Nintendo,Action,Fantasy,Kirby,NaN,NaN,NaN,88,Kirby and the Rainbow Paintbrush,88
150,Warhammer Age of Sigmar: Storm Ground,Warhammer: Age of Sigmar - Stormground,2021,Gasket Games,Focus Home Interactive,Strategy,Fantasy,Warhammer,NaN,NaN,NaN,85,Warhammer Age of Sigmar: Storm Ground,85
156,Dragon Quest X: The Five Awakening Races,Dragon Quest X: Mezameshi Itsutsu no Shuzoku O...,"August 2, 2012",Armor Project,Square Enix,"[Role-Playing, MMORPG]",Fantasy,Dragon Quest,NaN,NaN,NaN,56,Dragon Quest X: The Five Awakening Races,56
167,Pokemon X Version,Pokémon X/Y,"October 12, 2013","Game Freak, Inc.",Nintendo,Role-Playing,Fantasy,Pokémon,"[Nintendo, The Pokémon Company]",NaN,NaN,67,Pokemon X Version,67
284,Puzzles & Dragons Z + Puzzles & Dragons Super M,Puzzle & Dragons Z + Puzzle & Dragons: Super M...,"May 8, 2015",GungHo Online Entertainment,Nintendo,"[Compilation, Puzzle]",Fantasy,Mario,NaN,NaN,NaN,75,Puzzles & Dragons Z + Puzzles & Dragons Super M,75
288,Disgaea 5 Complete,Disgaea 5: Alliance of Vengeance,"March 26, 2015",Nippon Ichi Software Inc.,Nippon Ichi Software Inc.,"[Strategy, Role-Playing]",Fantasy,Disgaea,"[Nippon Ichi Software Inc., NIS America, Inc.]",NaN,NaN,67,Disgaea 5 Complete,67


In [323]:
df[df.Developer.isnull()]

,vgt_name,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses,fuzzy_ratio
57,Wipeout: Create & Crash,Wipeout: Create & Crash,"October 15, 2013",None,Activision,None,None,Wipeout,NaN,NaN,NaN,100
96,Far Cry 6,Far Cry 6,"May 25, 2021",None,Ubisoft Entertainment,First-Person Shooter,Post-Apocalyptic,Far Cry,NaN,NaN,NaN,100
118,Just Dance 2021,Just Dance 2021,"November 12, 2020",None,Ubisoft Entertainment,Music/Rhythm,None,Just Dance,NaN,NaN,NaN,100
127,Criminal Girls 2: Party Favors,Criminal Girls 2: Party Favors,"September 20, 2016",None,Nippon Ichi Software Inc.,None,None,Criminal Girls,NaN,NaN,NaN,100
246,Destiny 2 Forsaken,After The End: Forsaken Destiny,"May 30, 2017",None,None,"[Adventure, Puzzle]",None,None,NaN,NaN,NaN,94
258,AGOS: A Game of Space,AGOS: A Game of Space,N/A,None,None,None,None,None,NaN,NaN,NaN,100
283,The Lost Child,Lost Child,"March 24, 2003",None,None,Adventure,Anime,None,NaN,NaN,NaN,100
430,Lalaloopsy: Carnival of Friends,Lalaloopsy,"November 8, 2011",None,Activision,Simulation,Abstract,None,NaN,NaN,NaN,100
443,Sports Party,Sports Party,"October 30, 2018",None,Ubisoft Entertainment,"[Sports, Minigame Collection]",None,Sports Party,NaN,NaN,NaN,100
479,Nobunaga's Ambition: Sphere of Influence,Nobunaga's Ambition: Sphere of Influence,"December 12, 2013",None,Koei Tecmo,Strategy,None,Nobunaga's Ambition,NaN,NaN,NaN,100


In [405]:
{'Destiny 2 Forsaken':'https://www.giantbomb.com/destiny-2/3030-52647/',
 'DOOM (Switch edition)':'https://www.giantbomb.com/doom/3030-20654/'
}

df[df.Publisher.isnull()]
# df.insert(23, 'Publisher', 'Milestone S.r.l.')

,vgt_name,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses,fuzzy_ratio,clean_title,fuzzy_ratio_2
246,Destiny 2 Forsaken,After The End: Forsaken Destiny,"May 30, 2017",None,None,"[Adventure, Puzzle]",None,None,NaN,NaN,NaN,94,Destiny 2 Forsaken,94
493,DOOM (Switch edition),Switch!,"December 11, 2010",None,None,Puzzle,Abstract,None,NaN,NaN,NaN,100,DOOM,0
874,Terraria 2,Terraria 2,N/A,Re-Logic,None,None,None,Terraria,NaN,NaN,NaN,100,Terraria 2,100
983,Exomecha,Exomecha,N/A,TwistedRed,None,First-Person Shooter,Sci-Fi,None,NaN,NaN,NaN,100,Exomecha,100
1123,Jett: The Far Shore,Jett: The Far Shore,2021,Superbrothers,None,Adventure,Sci-Fi,None,NaN,NaN,NaN,100,Jett: The Far Shore,100
1984,Payday 3,Payday 3,None,Starbreeze AB,None,"First-person shooter,Tactical shooter,Stealth",None,Payday,None,None,None,100,Payday 3,100


In [444]:
# df[df.Publisher.isnull()]
# df.loc[701,'Publisher'] = 'Standfast Interactive'
df.loc[1921,'Developer'] = 'Electronic Arts Salt Lake'

# RedBedlam Developer and Standfast Interactive Publisher

In [446]:
rescrape = {'Destiny 2 Forsaken':'https://www.giantbomb.com/destiny-2/3030-52647/',
 'DOOM (Switch edition)':'https://www.giantbomb.com/doom/3030-20654/'
}

In [447]:
driver = webdriver.Chrome(executable_path='/Users/elena.morais/Documents/chromedriver_2')

for title in rescrape.keys():
    print(title)
    driver.get(rescrape[title])

    time.sleep(random.randint(2,5))
    get_game_details_2(title,driver,games_dict)
    
save_pickle('giant_bomb_updated_game_info.pkl',games_dict)

Destiny 2 Forsaken
DOOM (Switch edition)


'giant_bomb_updated_game_info.pkl is saved'

In [448]:
df = pd.DataFrame.from_dict(games_dict, orient='index')
df = df.reset_index().rename(columns={'index':'vgt_name'})
df['fuzzy_ratio'] = df.apply(get_fuzzy_match, axis=1)
df['clean_title'] = df.vgt_name.apply(lambda title: title.split('(')[0])
df['fuzzy_ratio_2'] = df.apply(get_fuzzy_match_2, axis=1)

In [449]:
df

,vgt_name,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses,fuzzy_ratio,clean_title,fuzzy_ratio_2
0,Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,NaN,NaN,NaN,100,Dragon Quest XI S: Echoes of an Elusive Age - D,100
1,TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,NaN,NaN,NaN,100,TANK! TANK! TANK!,100
2,Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,NaN,NaN,NaN,100,Road Rage,100
3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,[],None,Knack,NaN,NaN,NaN,83,Knack 2,83
4,Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision,"[Role-Playing, First-Person Shooter]",Sci-Fi,Destiny,NaN,NaN,NaN,100,Destiny: The Taken King,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,Dark Rose Valkyrie,Dark Rose Valkyrie,"July 21, 2016","Compile Heart, Inc.",Idea Factory International,Role-Playing,Alternate Historical,None,NaN,NaN,NaN,100,Dark Rose Valkyrie,100
2026,Dead Space 3,Dead Space 3,"February 5, 2013",Visceral Games,EA Games,"[Action, Shooter]",Horror,Dead Space,NaN,NaN,NaN,100,Dead Space 3,100
2027,Olympic Games Tokyo 2020,Olympic Games Tokyo 2020: The Official Video Game,"July 24, 2019",Sega,Sega,Sports,None,Olympic Games,NaN,NaN,NaN,100,Olympic Games Tokyo 2020,100
2028,Windbound,Windbound,"August 28, 2020",5 Lives Studios,Deep Silver,"[Action, Role-Playing]",Fantasy,None,NaN,NaN,NaN,100,Windbound,100


In [451]:
df[df.Publisher.isnull()].shape

(29, 14)

In [505]:
df[df.Publisher.isnull()]

,vgt_name,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses,fuzzy_ratio,clean_title,fuzzy_ratio_2,The Sims 3: Dragon Valley
760,Quantum Error,Quantum Error,N/A,TeamKill Media,None,First-Person Shooter,Horror,None,NaN,NaN,NaN,100,Quantum Error,100,NaN
874,Terraria 2,Terraria 2,N/A,None,None,None,None,Terraria,NaN,NaN,NaN,100,Terraria 2,100,NaN
983,Exomecha,Exomecha,N/A,TwistedRed,None,First-Person Shooter,Sci-Fi,None,NaN,NaN,NaN,100,Exomecha,100,NaN
1899,System Shock 3,System Shock 3,N/A,OtherSide Entertainment,None,Role-Playing,Sci-Fi,System Shock,NaN,NaN,NaN,100,System Shock 3,100,NaN
1984,Payday 3,Payday 3,N/A,Starbreeze AB,None,"First-person shooter,Tactical shooter,Stealth",None,Payday,None,None,None,100,Payday 3,100,NaN


In [504]:
df.loc[1984,'First release date'] = 'N/A'
# df.loc[1915,'Developer'] = 'Maxis Redwood Shores, EA Mobile, Edge of Reality'

In [506]:
save_pickle('updated_tables_vgt_giantbomb.pkl',df)

'updated_tables_vgt_giantbomb.pkl is saved'

In [511]:
df.columns

Index(['vgt_name', 'Name', 'First release date', 'Developer', 'Publisher',
       'Genre', 'Theme', 'Franchise', 'Publisher_extra', 'Platfrom',
       'Aliasses', 'fuzzy_ratio', 'clean_title', 'fuzzy_ratio_2',
       'The Sims 3: Dragon Valley'],
      dtype='object')

In [512]:
df = df[['vgt_name', 'Name', 'First release date', 'Developer', 'Publisher',
       'Genre', 'Theme', 'Franchise','Publisher_extra', 'Platfrom',
       'Aliasses', 'fuzzy_ratio', 'clean_title', 'fuzzy_ratio_2']]

In [524]:
df.Developer = df.Developer.apply(lambda dev: ','.join(dev) if type(dev) == type([]) else dev)
df.Publisher = df.Publisher.apply(lambda pub: ','.join(pub) if type(pub) == type([]) else pub)
df.Genre = df.Genre.apply(lambda g: ','.join(g) if type(g) == type([]) else g)
df.Publisher_extra = df.Publisher_extra.apply(lambda P: ','.join(P) if type(P) == type([]) else P)
df.Platfrom = df.Platfrom.apply(lambda P: ','.join(P) if type(P) == type([]) else P)
df.Aliasses = df.Aliasses.apply(lambda A: ','.join(A) if type(A) == type([]) else A)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [525]:
df

,vgt_name,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra,Platfrom,Aliasses,fuzzy_ratio,clean_title,fuzzy_ratio_2
0,Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,NaN,NaN,NaN,100,Dragon Quest XI S: Echoes of an Elusive Age - D,100
1,TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,NaN,NaN,NaN,100,TANK! TANK! TANK!,100
2,Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,NaN,NaN,NaN,100,Road Rage,100
3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,,None,Knack,NaN,NaN,NaN,83,Knack 2,83
4,Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision,"Role-Playing,First-Person Shooter",Sci-Fi,Destiny,NaN,NaN,NaN,100,Destiny: The Taken King,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,Dark Rose Valkyrie,Dark Rose Valkyrie,"July 21, 2016","Compile Heart, Inc.",Idea Factory International,Role-Playing,Alternate Historical,None,NaN,NaN,NaN,100,Dark Rose Valkyrie,100
2026,Dead Space 3,Dead Space 3,"February 5, 2013",Visceral Games,EA Games,"Action,Shooter",Horror,Dead Space,NaN,NaN,NaN,100,Dead Space 3,100
2027,Olympic Games Tokyo 2020,Olympic Games Tokyo 2020: The Official Video Game,"July 24, 2019",Sega,Sega,Sports,None,Olympic Games,NaN,NaN,NaN,100,Olympic Games Tokyo 2020,100
2028,Windbound,Windbound,"August 28, 2020",5 Lives Studios,Deep Silver,"Action,Role-Playing",Fantasy,None,NaN,NaN,NaN,100,Windbound,100


In [526]:
df_for_table = df[['vgt_name', 'Name', 'First release date', 'Developer', 'Publisher',
       'Genre', 'Theme', 'Franchise','Publisher_extra', 'Platfrom',
       'Aliasses']]

In [529]:
df_for_table.to_sql('Giantbomb_vgt_title_publisher_updates',if_exists='replace',con=DB_CONNECTION)

In [211]:
#clean up the giant bomb scraper for top publishers:

In [212]:
db_connection = query.get_db_connection('../../../.supreme_db_creds.pkl',
                                            'Sandbox_Elena_Morais',
                                            dbhost='centaur')
giantbomb_table = pd.read_sql("SELECT * FROM Giantbomb_vgt_title_publisher_updates_new",db_connection)

In [263]:
def get_top_pubs(pub,pubs_extra):
    if pubs_extra == None:
        return pub
    elif re.findall(r"ea ",pubs_extra.lower()) == ['ea ']:
        return "Electronic Arts, Inc."
    elif pubs_extra.lower().find("electronic arts") != -1:
        return "Electronic Arts, Inc."
    elif pubs_extra.lower().find("ubisoft") != -1:
        return "Ubisoft"
    elif re.findall(r"(take)\-?\s?(two)",pubs_extra.lower()) == [('take', 'two')]:
        return "Take Two Interactive Software, Inc."
    elif pubs_extra.lower().find("activision") != -1:
        return "Activision Blizzard"
    elif pubs_extra.lower().find("blizzard") != -1:
        return "Activision Blizzard"
    else:
        return pub

        

# giantbomb_table.Publisher = 
giantbomb_table["Publisher_new"] = giantbomb_table.apply(lambda row: get_top_pubs(row['Publisher'],row['Publisher_extra']),axis=1)

In [268]:
def get_top_pubs_2(pub):
    if pub == None:
        return pub
    elif re.findall(r"ea ",pub.lower()) == ['ea ']:
        return "Electronic Arts, Inc."
    elif pub.lower().find("electronic arts") != -1:
        return "Electronic Arts, Inc."
    elif pub.lower().find("ubisoft") != -1:
        return "Ubisoft"
    elif re.findall(r"(take)\-?\s?(two)",pub.lower()) == [('take', 'two')]:
        return "Take Two Interactive Software, Inc."
    elif pub.lower().find("activision") != -1:
        return "Activision Blizzard"
    elif pub.lower().find("blizzard") != -1:
        return "Activision Blizzard"
    else:
        return pub

giantbomb_table.Publisher_new = giantbomb_table.Publisher_new.apply(lambda pub: get_top_pubs_2(pub))

In [274]:
giantbomb_table = giantbomb_table[['level_0', 'index', 'Name', 'First release date', 'Developer',
       'Publisher_new', 'Genre', 'Theme', 'Franchise','Publisher_extra']]

In [277]:
giantbomb_table.rename(columns={'Publisher_new':'Publisher'},inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [280]:
giantbomb_table.to_sql('Giantbomb_vgt_title_publisher_updates_new',if_exists='replace',con=db_connection)


ValueError: duplicate name in index/columns: cannot insert level_0, already exists

In [282]:
giantbomb_table = giantbomb_table[['index', 'Name', 'First release date', 'Developer',
       'Publisher', 'Genre', 'Theme', 'Franchise', 'Publisher_extra']]

Index(['level_0', 'index', 'Name', 'First release date', 'Developer',
       'Publisher', 'Genre', 'Theme', 'Franchise', 'Publisher_extra'],
      dtype='object')

In [278]:
giantbomb_table

,level_0,index,Name,First release date,Developer,Publisher,Genre,Theme,Franchise,Publisher_extra
0,0,Dragon Quest XI S: Echoes of an Elusive Age - D,Dragon Quest XI: Echoes of an Elusive Age,"July 29, 2017",Toylogic,Square Enix,Role-Playing,Fantasy,Dragon Quest,None
1,1,TANK! TANK! TANK!,Tank! Tank! Tank!,"October 5, 2009",Bandai Namco Entertainment,Bandai Namco Entertainment,Vehicular Combat,Sci-Fi,None,None
2,2,Road Rage,Road Rage,"November 14, 2017",Team 6 Studios,Maximum Games,Driving/Racing,None,None,None
3,3,Knack 2,Knack II,"September 5, 2017",SIE Japan Studio,Sony Interactive Entertainment,"Action,Platformer",None,Knack,None
4,4,Destiny: The Taken King,Destiny,"September 9, 2014",Bungie,Activision Blizzard,"Role-Playing,First-Person Shooter",Sci-Fi,Destiny,"Activision,Sony Interactive Entertainment,Bungie"
...,...,...,...,...,...,...,...,...,...,...
2028,2028,Mechs vs. Minions,Mechs vs. Minions,None,"Riot Games Inc.,Stone Librande",Riot Games Inc.,Tabletop game,None,League of Legends,None
2029,2029,European Ship Simulator,European Ship Simulator,"Nov 4, 2016",Excalibur,Excalibur Publishing,Simulation,None,None,None
2030,2030,Puzzler: Brain Games,Puzzler: Brain Games,"April 23, 2013",Maximum Games,Nintendo,Puzzle,None,Puzzler,None
2031,2031,Payday 3,Payday 3,None,Starbreeze AB,None,"First-person shooter,Tactical shooter,Stealth",None,Payday,None


In [230]:
# dict_pubs = {}

# for p in giantbomb_table.Publisher_extra.to_list():
#     if p == None:
#         pass
#     else:
#         pubs = p.split(',')
#         for pub in pubs:
#             if pub in dict_pubs.keys():
#                 dict_pubs[pub] += 1
#             else:
#                 dict_pubs[pub] = 1
# giantbomb_table.Publisher.value_counts().iloc[:50]

In [251]:
tester = "rockstar games,take-two interactive software,sony,easy going,ea sports, electronic arts"
# re.findall(r",ea ",tester)
re.findall(r"(take)\-?\s?(two)",tester)

[('take', 'two')]

In [ ]:
fixed_pubs = {'Activision Blizzard':['Activision','Blizzard'],
              'Electronic Arts, Inc.':['EA Sports','Electronic Arts','Electronic Arts, Inc.','EA'],
              'Ubisoft':['Ubisoft Entertainment','Ubisoft San Francisco'],
              'Take Two Interactive Software, Inc.':['Take-Two Interactive Software']
}




Ubisoft Entertainment
Ubisoft San Francisco

Take-Two Interactive Software

EA Sports
Electronic Arts
Electronic Arts, Inc.
EA



Sony Interactive Entertainment
Sony Interactive Entertainment Europe
Sony Interactive Entertainment America

NIS America, Inc.

Curve Digital Entertainment
Curve Digital

Bandai Namco Entertainment
Bandai Namco Entertainment Europe
Namco

Level-5 Inc.

In [235]:
dict_pubs

{'Activision': 19,
 'Sony Interactive Entertainment': 21,
 'Bungie': 7,
 'Nippon Ichi Software Inc.': 15,
 'NIS America': 55,
 ' Inc.': 132,
 'Agatsuma Entertainment': 1,
 'Atlus U.S.A.': 37,
 'Degica Games': 1,
 'Nicalis': 4,
 'Idea Factory': 4,
 'Aksys Games': 23,
 'Plug in Digital': 3,
 'Nacon': 8,
 'Milestone s.r.l.': 2,
 'Daedalic Entertainment': 2,
 'Kalypso Media': 2,
 'Square Enix': 24,
 'Nintendo': 67,
 'GIANTS Software GmbH': 4,
 'Focus Home Interactive': 11,
 'Nihon Falcom Corp.': 7,
 'XSEED Games': 33,
 'Clouded Leopard Entertainment Inc.': 5,
 'Sony Interactive Entertainment Europe': 44,
 'Sony Interactive Entertainment America': 49,
 'Valve Corporation': 2,
 'Nexon Corporation': 2,
 'Perfect World Entertainment': 3,
 'Atlus Co.': 25,
 ' Ltd.': 48,
 'Sega': 14,
 'Aquaplus': 5,
 'Deep Silver': 17,
 'DMM Games': 3,
 'Shiravune': 2,
 'EA Sports': 6,
 'Electronic Arts': 7,
 '505 Games': 9,
 'Starbreeze Studios AB': 3,
 'Devolver Digital': 7,
 'Special Reserve Games': 6,
 'Rock

In [ ]:
top_publishers = ['Activision Blizzard','Electronic Arts, Inc.',
                  'Ubisoft','Take Two Interactive Software, Inc.','Microsoft Studios']

updated_publishers = {}

for item in publisher_dict.keys():
    if re.findall(r"^ea",publisher_dict[item][0].lower()) == ['ea']:
        updated_publishers[item] = 'Electronic Arts, Inc.'
    elif publisher_dict[item][0].lower().find('ubisoft') != -1:
        updated_publishers[item] = 'Ubisoft'
    elif  sum([publisher_dict[item][0].lower().find('activision'),publisher_dict[item][0].lower().find('blizzard')]) >= -1:
        updated_publishers[item] = 'Activision Blizzard'
    elif publisher_dict[item][0].lower().find('sony') != -1:
        updated_publishers[item] = 'Sony Interactive Entertainment'
    elif publisher_dict[item][0].lower().find('sega') != -1:
        updated_publishers[item] = 'Sega'
    elif publisher_dict[item][0].lower().find('namco') != -1 or publisher_dict[item][0].lower().find('bandai') != -1:
        updated_publishers[item] = 'Bandai Namco Entertainment'
    elif publisher_dict[item][0].lower().find('lucas') != -1:
        updated_publishers[item] = 'LucasArts'
    elif publisher_dict[item][0] == '(Archive) Ubi Soft':
        updated_publishers[item] = 'Ubisoft'
    else:
        for publisher in top_publishers:
            possible_titles = []
            if len(publisher_dict[item][0].split(' ')) < 2:
                sim = similar(publisher_dict[item][0], publisher.split(' ')[0])
            elif len(publisher.split(' ')) < 2:
                sim = similar(publisher_dict[item][0].split(' ')[0], publisher)
            else:
                sim = similar(publisher_dict[item][0], publisher)
            if sim > 0.68:
                updated_publishers[item] = publisher
                break
            else:
                updated_publishers[item] = publisher_dict[item][0]

In [ ]:
for 